In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/cookie_cats.csv')

In [ ]:
reg lwage exper expersq educ age kidslt6 kidsge6

In [3]:
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [4]:
cat = df.select_dtypes('object').columns
df = pd.get_dummies(df, columns = cat, drop_first = True)

In [6]:
df.head()

,userid,sum_gamerounds,retention_1,retention_7,version_gate_40
0,116,3,False,False,0
1,337,38,True,False,0
2,377,165,True,False,1
3,483,1,False,False,1
4,488,179,True,True,1


In [7]:
outcome = 'retention_7'
treatment = 'version_gate_40'
rest = list(df.drop([outcome, treatment, 'userid', 'retention_1'], axis = 1).columns)
#rest = ['exper','age', 'kidslt6', 'kidsge6']
df = df[[outcome] + [treatment] + rest]

In [8]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
#df = df.dropna()
df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')
print(df.shape)
x.head()

(90189, 3)


,sum_gamerounds
0,3.0
1,38.0
2,165.0
3,1.0
4,179.0


In [9]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:            retention_7   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     10.01
Date:                Mon, 05 Dec 2022   Prob (F-statistic):            0.00155
Time:                        02:20:41   Log-Likelihood:                -42850.
No. Observations:               90189   AIC:                         8.570e+04
Df Residuals:                   90187   BIC:                         8.572e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0082      0.003     -3.164      0.0

In [10]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:            retention_7   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     3820.
Date:                Mon, 05 Dec 2022   Prob (F-statistic):               0.00
Time:                        02:20:47   Log-Likelihood:                -39188.
No. Observations:               90189   AIC:                         7.838e+04
Df Residuals:                   90186   BIC:                         7.841e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0076      0.002     -3.036      0.0

In [11]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import XGBRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 8)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: retention_7
Treatment variable(s): ['version_gate_40']
Covariates: ['sum_gamerounds']
Instrument variable(s): None
No. Observations: 90189

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Columns: 3 entries, retention_7 to sum_gamerounds
dtypes: bool(1), int64(1), uint8(1)
memory usage: 880.9 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: retention_7
Treatment variable(s): ['version_gate_40']
Covariates: ['sum_gamerounds']
Instrument variable(s): None
No. Observations: 90189

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=8,

In [13]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: retention_7
Treatment variable(s): ['version_gate_40']
Covariates: ['sum_gamerounds']
Instrument variable(s): None
No. Observations: 90189

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Columns: 3 entries, retention_7 to sum_gamerounds
dtypes: bool(1), int64(1), uint8(1)
memory usage: 880.9 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: retention_7
Treatment variable(s): ['version_gate_40']
Covariates: ['sum_gamerounds']
Instrument variable(s): None
No. Observations: 90189

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=6,

In [14]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,-0.006,0.003,-1.982,0.047,-0.013,-0.0
